# Scop3P

A comprehensive database of human phosphosites within their full context. Scop3P integrates sequences (UniProtKB/Swiss-Prot), structures (PDB), and uniformly reprocessed phosphoproteomics data (PRIDE) to annotate all known human phosphosites. 

Scop3P, available at https://iomics.ugent.be/scop3p, presents a unique resource for visualization and analysis of phosphosites and for understanding of phosphosite structure–function relationships.

### Install Dependencies

In [2]:
%%capture
!jupyter labextension install jupyterlab_3dmol
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!pip install pandas matplotlib py3Dmol b2btools==3.0.7b2 nodejs

### Import required packages

In [1]:
%%capture
import requests, tempfile,json,sys
import pandas as pd 
from b2bTools import SingleSeq, constants
import py3Dmol
import ipywidgets as widgets

### Upload your peptide (tab delimited .txt) file in the next cell

In [16]:
wid=widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

display(wid)

In [17]:
wid.value


In [18]:
pepfile=pd.read_csv('peptidefile.txt',sep='\t',usecols=[1,2,4,5,6,7])
proteincol=input("Enter Protein ID column number: ")
proteincolname=pepfile.columns[int(proteincol)-1]
print ("Protein column name in provided file is: ", proteincolname)

In [19]:
pepgroup=pepfile.groupby(proteincolname)
proteinlist=list(set(pepfile[proteincolname].tolist()))
print ("Total number of proteins in your file: ", len(proteinlist))


In [8]:
def fetch_prot_sequence(accession):
    BASE_URL = f"http://uniprot.org/uniprotkb/{accession}.fasta"
    url = f'{BASE_URL}?accession={accession}'
    response = requests.get(url)
    if response.status_code == 200:
        raw_fasta_sequence = response.content.decode("utf-8")
    else:
        raw_fasta_sequence = ""
    
    lines = raw_fasta_sequence.split('\n')
    protein_id = str(lines[0])
    amino_acids = "".join([str(l) for l in lines[1:]])
    
    return protein_id, amino_acids

In [9]:
# def UPseqfetch(protseq,accsn,pep):
import re

def add_element(tmpdict, key, value):
    if key not in tmpdict:
        tmpdict[key] = []
    tmpdict[key].append(value)

def pepMapper(protDF):
    dataDF={}
    for idx, grp in pepgroup:
        protid=grp[proteincolname].tolist()[0]
        peplis=list(set(grp['Pep_seq'].tolist()))
        id_,protseq=fetch_prot_sequence(protid)
        for pep in peplis:
            for match in re.finditer(pep, protseq):
                newpos=[match.start()+1, match.end()+1]
                add_element(dataDF,protid,[pep,newpos[0],newpos[1]])
    return dataDF

In [20]:
mapped_peptides=pepMapper(pepgroup)
print ("Protein\t#Peptides")
print("\n".join("{}\t{}".format(key,len(value)) for key,value in mapped_peptides.items()))

In [21]:
structureID=input("Please enter the protein Id for 3D visualization: ")
peptides=mapped_peptides[structureID]

In [13]:

## Get alphaFold model for the protein
import urllib.request
AFurl="https://alphafold.ebi.ac.uk/files/AF-"
modelurl = f'{AFurl}{structureID}{"-F1-model_v6.pdb"}'
AFmodel = urllib.request.urlretrieve(modelurl,f'{structureID}{".pdb"}')


In [14]:
import itertools
def display_3D(peptides):
    view = py3Dmol.view()
    view.addModel(open((structureID+'.pdb'), 'r').read(),'pdb')
    allpos=[]
    colordict,commoncol={},{}
    view.setStyle({'cartoon': { 'color': 'silver' }})
#     view.addSurface(py3Dmol.VDW, {'opacity': 0.80, 'color': 'white' })
    for pep in peptides:
        for aapos in range(pep[1],pep[2]+1):
            allpos.append(list(range(pep[1],pep[2]+1)))
            
    mergedPos = list(itertools.chain.from_iterable(allpos))
    commonPos=list(set.intersection(*map(set, allpos))) 
    for aapos in mergedPos:
        colordict[aapos]='blue'
#         view.addSurface(py3Dmol.VDW, {'opacity': 0.6,'color':'red'},{'resi': [aapos]})
    for aapos1 in commonPos:
        commoncol[aapos1]='red'
#         view.addSurface(py3Dmol.VDW, {'opacity': 0.6,'color':'purple'},{'resi': [aapos1]})              
    print ("Common positions colored in Purple")    

    view.setStyle({'cartoon': {'colorscheme':{'prop':'resi','map':colordict}}})     
    view.setStyle({'cartoon': {'colorscheme':{'prop':'resi','map':commoncol}}})
    
    view.zoomTo()
    return view

In [22]:
display_3D(peptides)